In [27]:
import requests
import urllib3
import pandas as pd
from datetime import datetime
from datetime import time
from datetime import date

Interesting link: https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde
Strava API forum: https://groups.google.com/g/strava-api

In [28]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
payload = {
    'client_id': "75483",
    'client_secret': 'd7d1e5483201559ddf62c677ac12f6cf7847c5ae',
    'refresh_token': '75994cf8182a3b6a956390b5a4da3eba764f1d85',
    'grant_type': "refresh_token",
    'f': 'json',
}

In [29]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

Requesting Token...

Access Token = 6204604ec8f73039d1a75311d24c4d47577e5638



In [30]:
activites_url = "https://www.strava.com/api/v3/athlete/activities"

dataset=[]
pages_count=1

while pages_count>0:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': pages_count}
    my_dataset=requests.get(activites_url, headers=header, params=param).json()
    if my_dataset==[]:
        break
    elif my_dataset!=[]:
        dataset.append(my_dataset)
        pages_count+=1
dataset = [j for i in dataset for j in i]


In [31]:
activities = pd.json_normalize(dataset)

In [32]:
#Create new dataframe with only columns I care about
cols = ['name', 'upload_id', 'type', 'distance', 'moving_time',   
         'average_speed', 'max_speed','total_elevation_gain',
         'start_date_local'
       ]

activities = activities[cols]

#Break date into start time and date
activities['start_date_local']= [datetime.strptime(i,"%Y-%m-%dT%H:%M:%SZ") for i in activities['start_date_local']]

activities['start_time']=[time(hour=i.hour,minute=i.minute,second=i.second) for i in activities['start_date_local']]
activities['start_date_local'] =[date(year=i.year,month=i.month,day=i.day) for i in activities['start_date_local']]
activities.head(10)

,name,upload_id,type,distance,moving_time,average_speed,max_speed,total_elevation_gain,start_date_local,start_time
0,Afternoon Swim,7008640440,Swim,1200.0,1375,0.873,4.860,0.0,2022-01-27,17:31:05
1,Afternoon Run,7001539338,Run,9343.2,2721,3.434,4.708,96.7,2022-01-26,13:06:56
2,Afternoon Run,6991911925,Run,8309.9,2471,3.363,4.323,100.8,2022-01-24,17:35:41
3,Afternoon Swim,6969582248,Swim,1000.0,1133,0.883,1.325,0.0,2022-01-20,17:45:08
4,Afternoon Run,6965007702,Run,7097.5,2311,3.071,4.728,96.1,2022-01-19,17:46:35
5,Afternoon Swim,6958679758,Swim,1200.0,1361,0.882,6.872,0.0,2022-01-18,17:32:28
6,Evening Run,6953646395,Run,6223.4,2407,2.586,4.328,0.0,2022-01-17,19:40:03
7,Lunch Ride,6946126922,Ride,30408.5,7385,4.118,11.385,520.3,2022-01-16,11:19:59
8,Afternoon Run,6917922349,Run,4771.7,1489,3.205,4.139,51.2,2022-01-11,13:23:52
9,Evening Swim,6914427420,Swim,1200.0,1496,0.802,1.187,0.0,2022-01-10,18:42:22


In [33]:
activities['type'].value_counts()

Ride              170
Run               117
Swim               83
Hike               50
Workout             2
BackcountrySki      1
Name: type, dtype: int64

### Get the maps of the routes
https://developers.google.com/maps/documentation/utilities/polylineutility

In [550]:
activities["map.summary_polyline"]


KeyError: 'map.summary_polyline'